## MultiViewAtlas demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
import mudata
import anndata
import scanpy as sc
import pandas as pd
import numpy as np

import multi_view_atlas as mva

## Load fetal-immune dataset

In [4]:
datadir = '/nfs/team205/ed6/data/Fetal_immune/cellxgene_h5ad_files/scRNA_data/'
h5ad_files = [f for f in os.listdir(datadir) if f.endswith('embedding.h5ad')]
h5ad_files

['PAN.A01.v01.raw_count.20210429.HSC_PROGENITORS.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.NKT.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.PFI.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.LYMPHOID.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.MEM_PROGENITORS.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.STROMA.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.MYELOID_V2.embedding.h5ad',
 'PAN.A01.v01.raw_count.20210429.HSC_IMMUNE.embedding.h5ad']

In [44]:
## Define the hierarchy between views 
view_hierarchy = {
    'HSC_IMMUNE':{
        'HSC_PROGENITORS':None,
        'LYMPHOID':{
            'NKT':None
        },
        'MYELOID_V2':None,
        'MEM_PROGENITORS':None
    },
    'STROMA':None
}

In [7]:
# Load full atlas 
adata_full = sc.read_h5ad(os.path.join(datadir, 'PAN.A01.v01.raw_count.20210429.PFI.embedding.h5ad'), backed=True)

# Load each view
adata_dict = {}
adata_dict['full'] = adata_full
for v in mva.utils.get_views_from_structure(view_hierarchy):
    vdata = sc.read_h5ad(os.path.join(datadir, f'PAN.A01.v01.raw_count.20210429.{v}.embedding.h5ad'), backed=True)
    # adata_dict[v] = AnnData(obs=vdata.obs, obsm=vdata.obsm)
    adata_dict[v] = vdata

### Initialize MultiViewAtlas object

A MultiViewAtlas object can be initialized from a `MuData` object, storing the view hierarchy in `.uns`, or from an `AnnData` object of the full dataset, additionally storing an assignment of cells to views in `adata.obsm['view_assign']`. 

The `transition_rule` parameter defines which info should be used to subset cells from one parent view to a child view. The transition rule is set to be the same for all transitions during initialization, but it can be updated later.  

In [14]:
# make table assigning cells to views
view_assign = pd.DataFrame(index=adata_full.obs_names)
for v in adata_dict.keys():
    view_assign[v] = view_assign.index.isin(adata_dict[v].obs_names).astype(int)

view_assign.head()


,full,HSC_IMMUNE,HSC_PROGENITORS,LYMPHOID,NKT,MYELOID_V2,MEM_PROGENITORS,STROMA
FCAImmP7579224-ATTACTCTCGATGAGG,1,1,0,0,0,1,0,0
FCAImmP7579224-CAGCCGAGTACATCCA,1,1,0,0,0,0,1,0
FCAImmP7579224-TGCTACCTCATGTAGC,1,1,0,0,0,1,0,0
FCAImmP7579224-ACGGCCACAAGCTGAG,1,1,0,0,0,1,0,0
FCAImmP7579224-CTAATGGCACTGTGTA,1,1,0,1,0,0,0,0


In [45]:
# # Initialize from AnnData
# adata_full.uns['view_hierarchy'] = view_hierarchy.copy()
# adata_full.obsm['view_assign'] = view_assign.copy()
# mvatlas = mva.tl.MultiViewAtlas(adata_full, transition_rule='X_scvi')

# Initialize from mudata object
mdata = mudata.MuData(adata_dict)
mdata['full'].uns['view_hierarchy'] = view_hierarchy.copy()
mvatlas = mva.tl.MultiViewAtlas(mdata, transition_rule='X_scvi')

In [46]:
mvatlas

MultiViewAtlas object with view hierarchy:
	full:
	  HSC_IMMUNE:
	    HSC_PROGENITORS: null
	    LYMPHOID:
	      NKT: null
	    MEM_PROGENITORS: null
	    MYELOID_V2: null
	  STROMA: null
	
MuData object with n_obs × n_vars = 911873 × 268304
  obsm:	'view_assign'
  8 modalities
    full:	911873 x 33538
      obs:	'n_counts', 'n_genes', 'file', 'mito', 'doublet_scores', 'predicted_doublets', 'old_annotation_uniform', 'organ', 'Sort_id', 'age', 'method', 'donor', 'sex', 'Sample', 'scvi_clusters', 'is_maternal_contaminant', 'anno_lvl_2_final_clean', 'celltype_annotation'
      var:	'GeneID', 'GeneName', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'scvi_model_var'
      uns:	'leiden', 'scvi', 'umap', 'view_hierarchy'
      obsm:	'X_scvi', 'X_umap', 'view_assign'
      obsp:	'scvi_connectivities', 'scvi_distances'
    HSC_IMMUNE:	593203 x 0
      obs:	'n_counts', 'n_genes', 'file', 'mito', 'doublet_scores', 'predicted_doublets', 'old_annotation_uniform', 'organ', 'Sort_i

The `MultiViewAtlas` object stores a MuData object where modalities are different subsets (views) of the full dataset. The full gene expression matrix is stored only in the `full` view, while for all other views we store latent dimensions in `.obsm` and metadata in `.obs` only. We can attach gene expression values to a single view when needed accessing directly from the MultiViewAtlas object.

In [ ]:
mvatlas.mdata['HSC_IMMUNE'] # No gene expression matrix in .X
mvatlas['HSC_IMMUNE'] # With gene expression matrix in .X

The hierarchy between views is stored in `mvatlas.view_hierarchy`

In [47]:
mvatlas.view_hierarchy

{'full': {'HSC_IMMUNE': {'HSC_PROGENITORS': None,
   'LYMPHOID': {'NKT': None},
   'MYELOID_V2': None,
   'MEM_PROGENITORS': None},
  'STROMA': None}}

The information used to transition from one parent view to a child view is stored in `mvatlas.view_transition_rule`. The transition rule can be a latent space (a slot in `.obsm`) or cell-level metadata (a column in `.obs`). If the transition rule is a latent space, cells in one view are assigned to the next view by similarity in that latent space. If the transition rule is cell-level metadata, cells in one view are assigned to the next view based on labels for that column in obs.

In [23]:
mvatlas.view_transition_rule

,full,HSC_IMMUNE,HSC_PROGENITORS,LYMPHOID,NKT,MYELOID_V2,MEM_PROGENITORS,STROMA
full,NaN,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi
HSC_IMMUNE,X_scvi,NaN,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,NaN
HSC_PROGENITORS,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
LYMPHOID,X_scvi,X_scvi,NaN,NaN,X_scvi,NaN,NaN,NaN
NKT,X_scvi,X_scvi,NaN,X_scvi,NaN,NaN,NaN,NaN
MYELOID_V2,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
MEM_PROGENITORS,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
STROMA,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can set different rules for different transitions. For example we can set the split from Lymphoid to NKT view to be based on annotation labels, rather than position in the latent space (__N.B. for now I haven't implemented any check that the rule is followed, if the atlas is already initialized!__)

In [27]:
mvatlas.set_transition_rule(parent_view='LYMPHOID', child_view='NKT', transition_rule='anno_lvl_2_final_clean')
mvatlas.view_transition_rule

,full,HSC_IMMUNE,HSC_PROGENITORS,LYMPHOID,NKT,MYELOID_V2,MEM_PROGENITORS,STROMA
full,NaN,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi
HSC_IMMUNE,X_scvi,NaN,X_scvi,X_scvi,X_scvi,X_scvi,X_scvi,NaN
HSC_PROGENITORS,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
LYMPHOID,X_scvi,X_scvi,NaN,NaN,anno_lvl_2_final_clean,NaN,NaN,NaN
NKT,X_scvi,X_scvi,NaN,anno_lvl_2_final_clean,NaN,NaN,NaN,NaN
MYELOID_V2,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
MEM_PROGENITORS,X_scvi,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN
STROMA,X_scvi,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can add new splits to a MultiViewAtlas object as we go, making a new assignment table. For example, we might want to split the stromal compartment by organ classes.

In [48]:
assign_dict = {
    'hematopoietic_tissue':['BM', "LI", "YS"],
    'lymphoid_tissue':['TH', "MLN", "SP"],
    'peripheral_tissue':['SK', "GU", 'KI']
}

transition_rule = 'organ'
parent_view = 'STROMA'

assign_tab = np.vstack([np.where(mvatlas.mdata[parent_view].obs[transition_rule].isin(assign_dict[k]), 1, 0) for k in assign_dict.keys()]).T
assign_tab = pd.DataFrame(assign_tab, columns = assign_dict.keys(), index = mvatlas.mdata[parent_view].obs_names)
mvatlas.update_views(parent_view='STROMA', child_assign_tab=assign_tab, transition_rule=transition_rule)

ValueError: transition_rule organ not found in .obsm or .obs

In [50]:
# pd.json_normalize(mvatlas.view_hierarchy).columns
mvatlas.mdata.obs

,full:n_counts,full:n_genes,full:file,full:mito,full:doublet_scores,full:predicted_doublets,full:old_annotation_uniform,full:organ,full:Sort_id,full:age,...,STROMA:Sort_id,STROMA:age,STROMA:method,STROMA:donor,STROMA:sex,STROMA:Sample,STROMA:scvi_clusters,STROMA:is_maternal_contaminant,STROMA:anno_lvl_2_final_clean,STROMA:celltype_annotation
FCAImmP7579224-ATTACTCTCGATGAGG,61563.0,6117,FCAImmP7579224,0.036321,0.087221,False,NaN,SK,CD45P,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FCAImmP7579224-CAGCCGAGTACATCCA,61511.0,6658,FCAImmP7579224,0.051909,0.125320,False,NaN,SK,CD45P,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FCAImmP7579224-TGCTACCTCATGTAGC,54545.0,6485,FCAImmP7579224,0.045284,0.164087,False,NaN,SK,CD45P,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FCAImmP7579224-ACGGCCACAAGCTGAG,51992.0,5768,FCAImmP7579224,0.040083,0.092437,False,NaN,SK,CD45P,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FCAImmP7579224-CTAATGGCACTGTGTA,51305.0,5492,FCAImmP7579224,0.046467,0.176471,False,NaN,SK,CD45P,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FCAImmP7803042-AGTGGGAGTCGACTGC,2015.0,865,FCAImmP7803042,0.032258,0.057283,False,NaN,SK,CD45N,14.0,...,CD45N,14.0,5GEX,F51,female,F51_SK_CD45N_FCAImmP7803042,29,False,FIBROBLAST_XIII,FIBROBLAST_XIII
FCAImmP7803042-CCATTCGTCAACACAC,2013.0,1095,FCAImmP7803042,0.034277,0.019263,False,FIBROBLAST,SK,CD45N,14.0,...,CD45N,14.0,5GEX,F51,female,F51_SK_CD45N_FCAImmP7803042,22,False,FIBROBLAST_V,FIBROBLAST_V
FCAImmP7803042-CGTAGCGAGTGACATA,2011.0,789,FCAImmP7803042,0.073098,0.098999,False,NaN,SK,CD45N,14.0,...,CD45N,14.0,5GEX,F51,female,F51_SK_CD45N_FCAImmP7803042,1,False,FIBROBLAST_I,FIBROBLAST_I
FCAImmP7803042-TGGCCAGCAATGAAAC,2002.0,1041,FCAImmP7803042,0.043457,0.017828,False,FIBROBLAST,SK,CD45N,14.0,...,CD45N,14.0,5GEX,F51,female,F51_SK_CD45N_FCAImmP7803042,2,False,FIBROBLAST_IV,FIBROBLAST_IV
